In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

In [2]:
spark = SparkSession.builder \
    .appName("Process Silver") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/07 19:01:24 WARN Utils: Your hostname, DESKTOP-DETHDJJ, resolves to a loopback address: 127.0.1.1; using 192.168.15.99 instead (on interface wifi0)
25/06/07 19:01:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/07 19:01:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
read_path = "/mnt/c/Users/LENOVO WINDOWS 10/Desktop/Solano/repos_sol/brewery_chalenge/datalake/bronze/raw_breweries.json"
write_path = "/mnt/c/Users/LENOVO WINDOWS 10/Desktop/Solano/repos_sol/brewery_chalenge/datalake/silver/silver_breweries"

In [5]:
%pwd

'/mnt/c/Users/LENOVO WINDOWS 10/Desktop/Solano/repos_sol'

In [4]:
raw_df = spark.read.json(read_path, multiLine=True)

In [5]:
raw_df.show()

+--------------------+---------+---------+------------+--------------+-------------+--------------------+---------------+----------------+--------------------+------------+-----------+-------------+--------------+--------------------+--------------------+
|           address_1|address_2|address_3|brewery_type|          city|      country|                  id|       latitude|       longitude|                name|       phone|postal_code|        state|state_province|              street|         website_url|
+--------------------+---------+---------+------------+--------------+-------------+--------------------+---------------+----------------+--------------------+------------+-----------+-------------+--------------+--------------------+--------------------+
|      1716 Topeka St|     NULL|     NULL|       micro|        Norman|United States|5128df48-79fc-4f0...|    35.25738891|    -97.46818222|    (405) Brewing Co|  4058160490| 73069-8224|     Oklahoma|      Oklahoma|      1716 Topeka S

In [6]:
raw_df.printSchema()

root
 |-- address_1: string (nullable = true)
 |-- address_2: string (nullable = true)
 |-- address_3: string (nullable = true)
 |-- brewery_type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- street: string (nullable = true)
 |-- website_url: string (nullable = true)



In [7]:
silver_df = raw_df.select(
    col("id").alias("brewery_id"),
    col("name").alias("brewery_name"),
    "brewery_type",
    "city",
    "state",
    "country"
)

In [8]:
silver_df.show()

+--------------------+--------------------+------------+--------------+-------------+-------------+
|          brewery_id|        brewery_name|brewery_type|          city|        state|      country|
+--------------------+--------------------+------------+--------------+-------------+-------------+
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|        Norman|     Oklahoma|United States|
|9c5a66c8-cc13-416...|    (512) Brewing Co|       micro|        Austin|        Texas|United States|
|34e8c68b-6146-453...|1 of Us Brewing C...|       micro|Mount Pleasant|    Wisconsin|United States|
|6d14b220-8926-452...|10 Barrel Brewing Co|       large|          Bend|       Oregon|United States|
|e2e78bd8-80ff-4a6...|10 Barrel Brewing Co|       large|          Bend|       Oregon|United States|
|e432899b-7f58-455...|10 Barrel Brewing Co|       large|      Portland|       Oregon|United States|
|ef970757-fe42-416...|10 Barrel Brewing Co|       large|     San Diego|   California|United States|


In [9]:
silver_df.write.mode("overwrite").partitionBy('country','brewery_type').parquet(write_path)